Advent of Code, Day 3

As usual, we'll start with reading in our data file. In this case, the only initial manipulation we'll need is to strip the trailing whitespace.

In [140]:
with open('data.csv') as f:
    lines = [l.strip() for l in f.readlines()]

Next, we'll begin preparing our data structure. We don't necessarily need to know which rucksack belongs to which Elf, but at this point, we can presume that's a likely requirement, so let's plan ahead. We'll employ basic string splicing and set theory to quickly narrow in on what item is the common item between the rucksack compartments.

In [141]:
elv_id = 0
rucksacks = {}

for l in lines:
    rucksacks[elv_id] = frozenset(l[:len(l)//2]).intersection(l[len(l)//2:])
    elv_id +=1

{0: frozenset({'l'}),
 1: frozenset({'j'}),
 2: frozenset({'q'}),
 3: frozenset({'j'}),
 4: frozenset({'Z'}),
 5: frozenset({'w'}),
 6: frozenset({'g'}),
 7: frozenset({'l'}),
 8: frozenset({'b'}),
 9: frozenset({'n'}),
 10: frozenset({'s'}),
 11: frozenset({'j'}),
 12: frozenset({'H'}),
 13: frozenset({'b'}),
 14: frozenset({'T'}),
 15: frozenset({'Z'}),
 16: frozenset({'W'}),
 17: frozenset({'N'}),
 18: frozenset({'z'}),
 19: frozenset({'s'}),
 20: frozenset({'J'}),
 21: frozenset({'m'}),
 22: frozenset({'V'}),
 23: frozenset({'j'}),
 24: frozenset({'S'}),
 25: frozenset({'m'}),
 26: frozenset({'p'}),
 27: frozenset({'N'}),
 28: frozenset({'g'}),
 29: frozenset({'J'}),
 30: frozenset({'P'}),
 31: frozenset({'m'}),
 32: frozenset({'Z'}),
 33: frozenset({'L'}),
 34: frozenset({'J'}),
 35: frozenset({'v'}),
 36: frozenset({'V'}),
 37: frozenset({'m'}),
 38: frozenset({'l'}),
 39: frozenset({'l'}),
 40: frozenset({'F'}),
 41: frozenset({'L'}),
 42: frozenset({'G'}),
 43: frozenset({'f'})

Next, we'll need to create our prioritization data. We can enumerate the integer values for each of the lower and upper case letter sets.

In [142]:
from string import ascii_lowercase, ascii_uppercase
lower_case_priorities = {v: i for i,v in enumerate(ascii_lowercase, 1)}
upper_case_priorities = {v: i for i,v in enumerate(ascii_uppercase, len(lower_case_priorities) + 1)}

priorities = lower_case_priorities | upper_case_priorities

Finally, we can go through and add up the values of each our our common rucksack items. Of note here is the unpacking of the `frozenset` in order to use the value as the lookup key for the priorities dict. Let's create a general function we can call to compute these values in case we need it again.

In [143]:
def compute_ruck_value(sacks, value=0):
    for r in sacks.values():
        item, = r
        value += priorities[item]

    return value

compute_ruck_value(rucksacks)

7793

Part Two

In part two, we'll need to make groups of three elves based on the order of the lines in the input file. itertools provides a recipe for just this case. We'll use this `batched` recipe to create our groups.

In [ ]:
import itertools

def batched(iterable, n):
    "Batch data into lists of length n. The last batch may be shorter."
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while (batch := list(itertools.islice(it, n))):
        yield batch

groups = batched(lines, 3)

With our groups ready to go, we next need to massage our rucksack contents into a form that will allow us to find the common item. We use list comprehension on our batch in order to make each item in the group a set so that we may easily find the intersection (i.e. the common item).

In [144]:
elv_groups = {}
elv_group_id = 0

for batch in groups:
    if elv_group_id in elv_groups:
        elv_groups[elv_group_id].append(batch)
    else:
        elv_groups[elv_group_id] = [set(b) for b in batch]
    elv_group_id += 1

{0: [{'B', 'r', 'p', 'P', 'M', 'T', 's', 'd', 'C', 'c', 'w', 'V', 't', 'l', 'G', 'h', 'L', 'R', 'f', 'D'}, {'t', 'Z', 'N', 'b', 'w', 'g', 'd', 'C', 'T', 'S', 'D', 'h', 'f', 'H', 'j', 'F', 'W', 'J'}, {'Z', 't', 'M', 'N', 'P', 'Q', 'g', 'm', 'B', 's', 'L', 'q', 'H', 'p', 'r', 'W', 'n', 'V'}], 1: [{'p', 'Q', 'm', 'H', 'j', 'J', 'z', 'd', 'C', 'c', 'F', 'V', 't', 'l', 'N', 'G', 'h', 'R', 'n'}, {'Z', 'N', 'g', 'C', 'G', 'h', 'L', 'r', 'F', 'W', 'J'}, {'P', 'M', 'v', 'b', 'l', 'N', 't', 'd', 's', 'D', 'B', 'f', 'q', 'H', 'w', 'z'}], 2: [{'t', 'v', 'P', 'N', 'w', 'g', 'd', 'C', 'T', 's', 'D', 'B', 'R', 'H', 'W', 'J', 'z'}, {'B', 'r', 'Z', 'Q', 'T', 'm', 's', 'j', 'z', 'd', 'F', 'W', 'l', 'G', 'h', 'L', 'D', 'f', 'R', 'n'}, {'Z', 'v', 'b', 'C', 'm', 'S', 'B', 'h', 'j', 'H', 'F', 'p', 'J'}], 3: [{'M', 'v', 'Z', 'w', 'g', 'd', 'C', 'c', 'S', 'G', 'D', 'J', 'B', 'f', 'p', 'W', 'n'}, {'P', 'Z', 'b', 'N', 'Q', 'd', 's', 'h', 'D', 'f', 'w', 'V'}, {'g', 'q', 'r', 'b', 'Q', 'T', 'm', 'S', 'j', 'J', 'c

The `badges` are the common item we will be looking for, using a combination of list expansion and dictionary comprehension to quickly get through all of our elv groups.

In [145]:
badges = {k:set.intersection(*v) for (k,v) in elv_groups.items()}

Finally, as expected, we can use our general compute ruck value function to calculate the sum of the contents based on their priority scores.

In [147]:
compute_ruck_value(badges)

2499